In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
new = pd.read_csv('/content/drive/MyDrive/newnot.csv')

In [ ]:
new

In [ ]:
def getDefaultQuestionIds():
  a=new.where(new['Concept']=='Default')
  a=a.dropna(subset=['Concept'])
  ans=a['Question No.']
  return a
  

In [ ]:
f=getDefaultQuestionIds()
f

In [ ]:
data=pd.DataFrame()
data=new.where(new['Concept']!='Default').dropna(subset=['Answer'])
data["Difficulty"] = pd.to_numeric(data["Difficulty"], downcast="float")
data['Question No.']=data['Question No.'].map(str)
data['Answer']=data['Answer'].map(str)

In [ ]:

#del data['Min. Time']
#del data['Max. Time']
# Setting minimum time allocated to each user on each question based on their difficulty
# this can changed by test maker accordingly
data['Min. Time']=data['Difficulty']*10
data['flag']=0
data

In [ ]:
newdata=data.sort_values(by=['Difficulty'])
newdata

In [ ]:
# setting up initial values of variables to be used
qno={}
difficulty={}
x_lower={}
x_upper={}
da={}
score={}

# Total concepts
concepts=data['Concept'].unique()
concepts

alpha=1
beta=1
flag=0

visited=np.zeros(data['Question'].size)
visited

# This is used for setting up how many questions to be asked from each concept
total=10
left=total%len(concepts)
questions_left={}
i=0
  
x_overall=0
for i in concepts: 
  questions_left[i]=int(total/len(concepts))
  if left>0:
    questions_left[i]+=1
    left-=1
  qno[i]=newdata['Question No.'].where(data['Concept']==i).dropna()
  difficulty[i]=newdata['Difficulty'].where(data['Concept']==i).dropna()
  da[i]=pd.DataFrame()
  da[i]['ques']=qno[i]
  da[i]['diff']=difficulty[i]
  score[i]=0
  x_upper[i]=difficulty[i].mean()
  x_overall+=x_upper[i]
  x_lower[i]=math.floor(x_upper[i])
  print(x_upper[i])
  print(x_lower[i])
  print(qno[i])
  print(difficulty[i])

x_overall/=len(concepts)
x_overall

In [ ]:
# This is rate at which x is changed which is based on sigmoid function or the logistic function
def update(alpha,x):
  return 1/(1+math.exp(alpha*-x))

In [ ]:
# This function is called when model is unable to find any question in given range
def ask2(m):
  k=0
  while k<10:
    j=m.sample()
    j=j.values.tolist()
    ind=data[data['Question No.']==j[0]].index

    c=data['Concept'][ind[0]]
    k=k+1
    if ((visited[ind[0]]==0)):
      visited[ind[0]]=1
      return str(data['Question No.'][ind[0]])
      break
  if k==10:
    x_upper[c]+=0.2
    x_lower[c]-=0.2
    m=da[c]['ques'].where((da[c]['diff']<=x_upper[c]) & (da[c]['diff']>=x_lower[c])).dropna()
    return ask2(m)

In [ ]:
def modifyQuestionQueue(queue,answers):
  i=1
  flag=0
  if i==0:
    queue=[]
    for c in concepts:
      alpha=0.5
      m=da[c]['ques'].where((da[c]['diff']<=x_upper[c]) & (da[c]['diff']>=x_lower[c])).dropna()
      x1=x_upper[c]
      x2=x_lower[c]
      j=ask2(m)
      x_upper[c]=x1
      x_lower[c]=x2
      queue.append(str(j))
      questions_left[c]-=1
      i+=1
    alpha=1
    return queue
  else:
    alpha=1
    if i>=total-1:
      queue=[]
      return queue
    for k in range(0,len(queue)):
      ind=data[data['Question No.']==queue[k]].index
      check=ind[0]
      c=data['Concept'][check]
      if answers[0][k]==data['Answer'][check]:
        if int(answers[1][k])<int(data['Min. Time'][check]):
            data['flag'][check]+=1
            flag+=1
        x_upper[c]/=update(alpha,x_upper[c])
        x_lower[c]=math.floor(x_upper[c])
        score[c]+=100*float(data['Difficulty'][check])
      else:
        x_upper[c]*=update(beta,x_upper[c])
        x_lower[c]=math.floor(x_upper[c])
    x_overall=0
    
    queue=[]
    c=random.choice(concepts)

    # asking given questions from each concept
    if questions_left[c]<=0:
      while questions_left[c]!=0:
        c=random.choice(concepts)
    questions_left[c]-=1
    for f in concepts:
      x_overall+=x_upper[f]
      x_overall/=len(concepts)
    m=da[c]['ques'].where((da[c]['diff']<=x_upper[c]) & (da[c]['diff']>=x_lower[c])).dropna()
    x1=x_upper[c]
    x2=x_lower[c]
    j=ask2(m)
    x_upper[c]=x1
    x_lower[c]=x2
    queue.append(str(j))
    questions_left[c]-=1
      # If there are many flags or value of x_overall too low ending test
    if flag>=0.3*total:
      queue=[]
      return queue
    if x_overall<1:
      queue=[]
      return queue
    i+=1
    return queue

In [ ]:
def Updatediff(answers):
  

In [ ]:
ask1=[]
queue=['1.0']
answers=[['3.0'],[30]]
ask1=modifyQuestionQueue(queue,answers)
ask1

In [ ]:
training_set=pd.read_csv('/content/drive/MyDrive/data1.csv')
training_set=training_set.drop(0,axis=0)
training_set

In [ ]:
training_set1=training_set.copy()

In [ ]:
training_set

In [ ]:
questions_left

In [ ]:
x_overall

In [ ]:
# Plotting graph
y=np.arange(1,11)
plt.plot(y,x_over,label='x_overall')
plt.plot(y,X[1],label='x for concept 1')
plt.plot(y,X[2],label='x for concept 2')
plt.legend()
plt.show()

In [ ]:
user_output=pd.DataFrame()

In [ ]:
user_output["answer"]=user_answer
user_output["question"]=index

In [ ]:
print(user_output)

In [ ]:
print(m)

In [ ]:
sim_options = {
    "name": "cosine",
    "user_based": False ,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

In [ ]:
training_set.shape

In [ ]:
training_set1

In [ ]:
ar=np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

In [ ]:
ar.shape

In [ ]:
training_set.loc[len(training_set.index)]=ar

In [ ]:
training_set

In [ ]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(training_set1)

In [ ]:
index

In [ ]:
data2=pd.DataFrame()

In [ ]:
j=0;
for i in index:
  data2[len(data2.columns)]=training_set.iloc[:,i-1]
  j=j+1

In [ ]:
data2

In [ ]:
visited

In [ ]:
y_train=pd.DataFrame()

In [ ]:
tex=range(18)
for i in tex:
  if visited[i]==0.:
    y_train[len(y_train.columns)]=training_set.iloc[:,i]


In [ ]:
data2

In [ ]:
x_train=data2

In [ ]:
y_train

In [ ]:
y_output=[]

In [ ]:
x_train

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', random_state=0)

In [ ]:
user_out=pd.DataFrame()

In [ ]:
user_out=user_output["answer"]

In [ ]:
data2

In [ ]:
data2

In [ ]:
user_out

In [ ]:
y_train

In [ ]:
user_out=user_out.to_numpy()

In [ ]:
user_out=np.reshape(user_out, ((1,len(user_out))), order='C')

In [ ]:
y_output=[]

In [ ]:
z12=range(len(y_train.columns))
for i in  z12:
  xtr=pd.DataFrame()
  xtr=y_train.iloc[:,i]
  model.fit(data2,xtr)
  z=model.predict(user_out)
  y_output.append(int(z))

In [ ]:
y_output

In [ ]:
allans=np.zeros(18)
j=0
for m in range(18):
  if user_quesans[m]==-1:
    user_quesans[m]=y_output[j]
    j+=1

In [ ]:
user_quesans


In [ ]:
training_set.loc[len(training_set.index)+1]=user_quesans

In [ ]:
training_set

In [ ]:
n=100

In [ ]:
data["Difficulty"] = pd.to_numeric(data["Difficulty"], downcast="float")

In [ ]:

for m in range(18):
  if user_quesans[m]==1:
    data['Difficulty'][m]-=1/n
  else:
    data['Difficulty'][m]+=1/n

In [ ]:
data

In [ ]:
score